In [1]:
# Thanks to Camille Rochefort-Boulanger for sharing this code with us!

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#  variables we will use
root = '/content/gdrive/My Drive/covid19 data/'
output_root = '/content/gdrive/My Drive/Covid Project/'

threshold = 0.005 #0.001 #1e-4 # 2e-5 # if diff in VAF btw waves >= threshold, target is 1, o.w. 0
vaf_remove_threshold = 0. #0.05 # set VAFs < vaf_remove_threshold to 0 

## DATA

### VAF matrix

In [4]:
VAF_FILE = root + 'Copy of samples_matrix.tsv'

In [5]:
# Load data
data = pd.read_csv(VAF_FILE, sep='\t')

In [6]:
data.head()

,sample_id,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,...,29739,29740,29741,29742,29743,29744,29745,29746,29747,29748,29749,29750,29751,29752,29753,29754,29755,29756,29757,29758,29759,29760,29761,29762,29763,29764,29765,29766,29767,29768,29769,29770,29771,29772,29773,29774,29775,29776,29777,29778
0,ERR4082713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ERR4082715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ERR4082716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ERR4082717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ERR4082718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data.shape

(11775, 29679)

In [8]:
# Remove Nan
data = data.dropna()
print(data.shape)

(11775, 29679)


# Targets

In [9]:
FIRST_WAVE_FILE = root + 'Copy of firstWave.frq'
SECOND_WAVE_FILE = root + 'Copy of secondWave.frq'

In [10]:
df1 = pd.read_csv(FIRST_WAVE_FILE, sep='\t', header=None, names=['chrom', 'pos', 'nb_obs', 'nb', 'A1_frq', 'A2_frq', 'A3_frq', 'A4_frq'])
df1 = df1[1:]
df1

,chrom,pos,nb_obs,nb,A1_frq,A2_frq,A3_frq,A4_frq
1,NC_045512.2,55,4,129294,A:0.999946,C:3.86716e-05,G:1.54686e-05,T:0
2,NC_045512.2,56,4,131057,G:0.999954,T:2.28908e-05,A:7.63027e-06,C:1.52605e-05
3,NC_045512.2,57,3,131091,A:0.999969,G:3.05132e-05,T:0,NaN
4,NC_045512.2,58,3,131132,T:0.999985,C:7.6259e-06,A:7.6259e-06,NaN
5,NC_045512.2,59,3,131167,C:0.999977,T:7.62387e-06,G:1.52477e-05,NaN
...,...,...,...,...,...,...,...,...
29778,NC_045512.2,29832,4,122970,T:0.999919,C:6.50565e-05,A:1.62641e-05,G:0
29779,NC_045512.2,29833,4,122913,A:0.999992,T:0,G:0,C:8.13584e-06
29780,NC_045512.2,29834,4,122779,T:0.99991,C:4.07236e-05,A:4.07236e-05,G:8.14472e-06
29781,NC_045512.2,29835,3,122585,C:0.999519,T:0.000481299,A:0,NaN


In [11]:
df2 = pd.read_csv(SECOND_WAVE_FILE, sep='\t', header=None, names=['chrom', 'pos', 'nb_obs', 'nb', 'A1_frq', 'A2_frq', 'A3_frq', 'A4_frq'])
df2 = df2[1:]
df2

,chrom,pos,nb_obs,nb,A1_frq,A2_frq,A3_frq,A4_frq
1,NC_045512.2,55,4,183019,A:0.999989,C:0,G:5.46391e-06,T:5.46391e-06
2,NC_045512.2,56,4,183863,G:0.999995,T:0,A:5.43883e-06,C:0
3,NC_045512.2,57,3,183882,A:0.999995,G:0,T:5.43827e-06,NaN
4,NC_045512.2,58,3,183905,T:0.999995,C:5.43759e-06,A:0,NaN
5,NC_045512.2,59,3,183963,C:0.999995,T:5.43588e-06,G:0,NaN
...,...,...,...,...,...,...,...,...
29778,NC_045512.2,29832,4,179960,T:0.99995,C:4.44543e-05,A:0,G:5.55679e-06
29779,NC_045512.2,29833,4,179934,A:0.999555,T:0,G:0.00043905,C:5.55759e-06
29780,NC_045512.2,29834,4,179853,T:0.9999,C:3.89207e-05,A:6.11611e-05,G:0
29781,NC_045512.2,29835,3,179424,C:0.999314,T:0.00067438,A:1.11468e-05,NaN


# Align Targets with Inputs

In [12]:
min_seq_num = int(data.keys()[1])
max_seq_num = int(data.keys()[-1])

In [13]:
# remove sequence positions which dont appear in data
df1.pos = df1.pos.astype(int)
df1 = df1[(df1.pos >= min_seq_num) & (df1.pos <= max_seq_num)]

df2.pos = df2.pos.astype(int)
df2 = df2[(df2.pos >= min_seq_num) & (df2.pos <= max_seq_num)]

## Get frequencies
Here we make arrays of nucleotide frequencies. Frequencies for each position are recorded for nucleotide A,C,T,G (in that arbitrary order). For example, cells C:0.99, T:0.01, G:0, A:0 become [0, 0.99, 0.01, 0]

In [14]:
# Empty arrays to be filled with A,C,T,G frq
first_wave = np.zeros((df1.shape[0], 4)) # nb of positions x 4 nucleotides
second_wave = np.zeros((df1.shape[0], 4))
alleles = ['A', 'C', 'T', 'G']

# Fill arrays
for i in range(df1.shape[0]):
  # --- First wave ---
  obs_alleles = []
  dict_frq = dict()
  if type(df1.iloc[i]['A1_frq']) == str:
    obs_alleles.append(df1.iloc[i]['A1_frq'][0])
    dict_frq[df1.iloc[i]['A1_frq'][0]] = np.float32(df1.iloc[i]['A1_frq'][2:])
  if type(df1.iloc[i]['A2_frq']) == str:
    obs_alleles.append(df1.iloc[i]['A2_frq'][0])
    dict_frq[df1.iloc[i]['A2_frq'][0]] = np.float32(df1.iloc[i]['A2_frq'][2:])
  if type(df1.iloc[i]['A3_frq']) == str:
    obs_alleles.append(df1.iloc[i]['A3_frq'][0])
    dict_frq[df1.iloc[i]['A3_frq'][0]] = np.float32(df1.iloc[i]['A3_frq'][2:])
  if type(df1.iloc[i]['A4_frq']) == str:
    obs_alleles.append(df1.iloc[i]['A4_frq'][0])
    dict_frq[df1.iloc[i]['A4_frq'][0]] = np.float32(df1.iloc[i]['A4_frq'][2:])
  
  # Add non observed nucleotides (frq = 0)
  for a in (set(alleles) - set(obs_alleles)):
    dict_frq[a] = 0.0
  
  # Add frq to np array
  first_wave[i-1] = [dict_frq['A'], dict_frq['C'], dict_frq['T'], dict_frq['G']]


  # --- Second wave ---
  obs_alleles = []
  dict_frq = dict()
  if type(df2.iloc[i]['A1_frq']) == str:
    obs_alleles.append(df2.iloc[i]['A1_frq'][0])
    dict_frq[df2.iloc[i]['A1_frq'][0]] = np.float32(df2.iloc[i]['A1_frq'][2:])
  if type(df2.iloc[i]['A2_frq']) == str:
    obs_alleles.append(df2.iloc[i]['A2_frq'][0])
    dict_frq[df2.iloc[i]['A2_frq'][0]] = np.float32(df2.iloc[i]['A2_frq'][2:])
  if type(df2.iloc[i]['A3_frq']) == str:
    obs_alleles.append(df2.iloc[i]['A3_frq'][0])
    dict_frq[df2.iloc[i]['A3_frq'][0]] = np.float32(df2.iloc[i]['A3_frq'][2:])
  if type(df2.iloc[i]['A4_frq']) == str:
    obs_alleles.append(df2.iloc[i]['A4_frq'][0])
    dict_frq[df2.iloc[i]['A4_frq'][0]] = np.float32(df2.iloc[i]['A4_frq'][2:])
  
  # Add non observed nucleotides (frq = 0)
  for a in (set(alleles) - set(obs_alleles)):
    dict_frq[a] = 0.0
  
  # Add frq to np array
  second_wave[i-1] = [dict_frq['A'], dict_frq['C'], dict_frq['T'], dict_frq['G']]

  if i % 1000 == 0:
    print(i, '/', df1.shape[0], 'completed.')

print(first_wave[0:5])
print(first_wave.shape)
print(second_wave[0:5])
print(second_wave.shape)

0 / 29678 completed.
1000 / 29678 completed.
2000 / 29678 completed.
3000 / 29678 completed.
4000 / 29678 completed.
5000 / 29678 completed.
6000 / 29678 completed.
7000 / 29678 completed.
8000 / 29678 completed.
9000 / 29678 completed.
10000 / 29678 completed.
11000 / 29678 completed.
12000 / 29678 completed.
13000 / 29678 completed.
14000 / 29678 completed.
15000 / 29678 completed.
16000 / 29678 completed.
17000 / 29678 completed.
18000 / 29678 completed.
19000 / 29678 completed.
20000 / 29678 completed.
21000 / 29678 completed.
22000 / 29678 completed.
23000 / 29678 completed.
24000 / 29678 completed.
25000 / 29678 completed.
26000 / 29678 completed.
27000 / 29678 completed.
28000 / 29678 completed.
29000 / 29678 completed.
[[2.93323992e-05 0.00000000e+00 2.19992999e-05 9.99948978e-01]
 [0.00000000e+00 9.99978006e-01 2.19982994e-05 0.00000000e+00]
 [7.33283014e-06 0.00000000e+00 9.99993026e-01 0.00000000e+00]
 [1.46631000e-05 0.00000000e+00 1.57627999e-03 9.98408973e-01]
 [0.0000000

In [15]:
np.save(output_root + 'first_wave.npy', first_wave)
np.save(output_root + 'second_wave.npy', second_wave)

## Make Target



In [16]:
first_wave = np.load(output_root + 'first_wave.npy')
second_wave = np.load(output_root + 'second_wave.npy')

In [17]:
# Compute difference
diff = second_wave - first_wave

In [18]:
max_diff_nucleotide = np.amax(np.abs(diff), axis=1)

In [19]:
target = max_diff_nucleotide.copy()
target[target > threshold] = 1 # 1 if > threshold
target[target <= threshold] = 0 # 0 if <= threshold

## Clean VAF Data

In [20]:
# VAF data
vaf = data[data.columns[1:]].to_numpy()
print(vaf.shape)

(11775, 29678)


In [21]:
vaf.shape, df1.shape # note that these now have the same dimension!

((11775, 29678), (29678, 8))

In [ ]:
pos = vaf <= vaf_remove_threshold
vaf[pos] = 0.0

## Remove Unneeded Columns!

In [ ]:
to_keep = (vaf.max(0) != 0)
np.save(output_root + 'snvs_to_keep_' + str(vaf_remove_threshold), to_keep)  # save which snvs were kept

In [ ]:
vaf = vaf[:, to_keep]
np.save(output_root + 'be_corrected_vaf_' + str(vaf_remove_threshold), vaf)

## Remove Targets which are no longer needed!

In [22]:
to_keep = np.load(output_root + 'Copy of snvs_to_keep_' + str(vaf_remove_threshold) + '.npy')

In [23]:
vaf.shape, target.shape, target[to_keep].shape

((11775, 29678), (29678,), (29678,))

In [24]:
target = target[to_keep] # remove SNVs which are no longer in our dataset

In [25]:
np.save(output_root + 'targets_' + str(threshold) + '_' + str(vaf_remove_threshold), target)

In [26]:
print('file saved to: ' + output_root + 'targets_' + str(threshold) + '_' + str(vaf_remove_threshold))

file saved to: /content/gdrive/My Drive/Covid Project/targets_0.005_0.0


In [27]:
target.mean()

0.00953568299750657